In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
# Carica Excel
df = pd.read_excel(r"../input/dati bollettino rev1.FI.xlsx")
df.query('DataSospetto >= "2008-01-01" & DataSospetto <= "2018-12-31"', inplace=True)

df['AnnoSospetto'] = pd.DatetimeIndex(df['DataSospetto']).year
df['MeseSospetto'] = pd.DatetimeIndex(df['DataSospetto']).month

# Cancella colonne non utili
df.drop(columns=['STATO_DEL_RECORD', 'AnnoFocolaioMinsal','NumFocolaioMinsal',
                 'DataConferma','DataEstinzione','DataInizioSintomi','CodComune',
                 'Malattia','NumCapiAbbattuti', 'NumCapiConSintomi', 'NumCapiDistrutti',
                 'NumCapiGuariti', 'NumCapiMalati', 'NumCapiMorti', 'NumCapiPresenti',
                 'TotaleMorti'], inplace=True)

# Visualizza campione
df.sample(3)

In [4]:
# Aggrega i dati ed esporta il dataset aggregato in CSV
df_grouped = df.groupby(['AnnoSospetto','CodIstat','Regione','Prov','Comune','Categorie']).count()["IdFocolaio"].reset_index(name="NumFocolai")
# df_grouped.to_csv('grouped.csv', index=False)

In [6]:
# Visualizza una porzione del dataset aggregato
# equidi_2018 = df_grouped.query("AnnoSospetto == '2018' & Categorie == 'EQUIDI'").sort_values(by='NumFocolai', ascending = False)
# equidi_2018.head(10)

In [14]:
# Carica lo shapefile dei Comuni e modifica i campi
gdf_comuni = gpd.read_file(r"../input/comuni_istat_2020_wgs84.shp")[['PRO_COM','geometry']]
gdf_comuni.rename(columns={'PRO_COM':'CodIstat'}, inplace=True)

In [18]:
# Join e creazione output
distribuzione = gdf_comuni.merge(df_grouped, on='CodIstat')
# distribuzione.sample(5)
# Esportazione
distribuzione.to_file(r"../output/distribuzione_per_comune_test.geojson", driver='GeoJSON')